In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1)
df1 = pd.read_csv('data_nov.csv').drop(['Unnamed: 0'], axis=1)

In [3]:
df

,название,цена,описание
0,"\n Porsche Cayman, 2013\n",\n 2 950 000 ₽\n,"\n 46 000 км, 2.7 AMT (275 л.с.), купе, зад..."
1,"\n Mitsubishi Eclipse, 2000\n",\n 470 000 ₽\n,"\n 170 000 км, 3.0 AT (208 л.с.), купе, пер..."
2,"\n Toyota Land Cruiser Prado, 2001\n",\n 850 000 ₽\n,"\n 185 200 км, 3.4 MT (185 л.с.), внедорожн..."
3,"\n KIA Rio, 2017\n",\n 659 000 ₽\n,"\n 18 300 км, 1.4 MT (107 л.с.), седан, пер..."
4,"\n Renault Duster, 2014\n",\n 580 000 ₽\n,"\n 200 000 км, 2.0 AT (135 л.с.), внедорожн..."
...,...,...,...
1757,"\n Chevrolet Niva, 2013\n",\n 420 000 ₽\n,"\n 49 000 км, 1.7 MT (80 л.с.), внедорожник..."
1758,"\n Chevrolet Niva, 2010\n",\n 290 000 ₽\n,"\n 114 800 км, 1.7 MT (80 л.с.), внедорожни..."
1759,"\n LADA Kalina, 2014\n",\n 325 000 ₽\n,"\n 65 000 км, 1.6 MT (87 л.с.), универсал, ..."
1760,"\n ВАЗ 2101, 1986\n",\n 40 000 ₽\n,"\n 38 500 км, 1.2 MT (59 л.с.), седан, задн..."


In [4]:
def f(df):
    for i in range(len(df['цена'])):
        if 'без скидки' in df['цена'][i]:
            df.drop([i], inplace=True)
    return df.reset_index(drop=True)

def price_prepare(prices):
    for i in range(len(prices)):
        price=int(prices[i].replace("₽", "").replace(" ",""))
        prices[i] = price
    return prices

def datasep(df,data,index):
    df['пробег'][index] = int(data[0].replace("км", "").replace(" ", "").strip())
    df['тип двигателя'][index] = data[4].strip()
    df['кузов'][index] = data[2].strip()
    df['привод'][index] = data[3].strip()
    mas = df['описание'][index].strip().split(",")
    w = mas[1].strip().split(' ')
    if len(w) == 4:
        df['объем двигателя'][index] = w[0].strip()
        df['коробка передач'][index] = w[1].strip().lower()
        df['л.с'][index] = w[2].replace("(", "").strip()
    else:
        df.drop([index], inplace=True)
    return df

def specification_prepare(df):
    df['пробег'] = ' '
    df['тип двигателя'] = ' '
    df['объем двигателя'] = ' '
    df['коробка передач'] = ' '
    df['кузов'] = ' '
    df['привод'] = ' '
    df['л.с'] = ' '
    df['состояние'] = ' '
    for i in range(len(df)):
        arr = df['описание'][i].strip().split(",")
        if len(arr) == 4:
            df = df.drop([i])
        elif len(arr) == 5:
            df['состояние'][i] = 'не битый'
            datasep(df,arr,i)
        else:
            arr.remove('Битый')
            df['состояние'][i] = 'битый'
            datasep(df,arr,i)
    df = df.drop(['описание'], axis=1)
    return df.reset_index(drop=True)

def name_prepare(names):
    names['название'] = names['название'].str.lower()
    pattern = re.compile(r'\w+')
    names['год'] = 1
    names['модель'] = ' '
    arr = names['название'].str.lower().str.strip().str.split(",")
    for i, j in enumerate(arr):
        names['год'][i] = j[1]
        word = pattern.findall(j[0])[0]
        model = names['название'][i].replace(word, '').replace(','+j[1], '').strip()
        names['название'][i] = word
        names['модель'][i] = model
    return names

def make_df(df1,df2):
    new_df = pd.concat([df1, df2],ignore_index=True)
    new_df = new_df.loc[new_df['год'] != ' до 1960']
    new_df['год'] = pd.to_numeric(new_df['год'])
    new_df['объем двигателя'] = pd.to_numeric(new_df['объем двигателя'])
    new_df['л.с'] = pd.to_numeric(new_df['л.с'])
    return new_df.reset_index(drop=True)

In [5]:
df1 = f(df1)
df['цена'] = price_prepare(df['цена'])
df = name_prepare(df)
df = specification_prepare(df)
df1['цена'] = price_prepare(df1['цена'])
df1 = name_prepare(df1)
df1 = specification_prepare(df1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A valu

In [6]:
df1

,название,цена,год,модель,пробег,тип двигателя,объем двигателя,коробка передач,кузов,привод,л.с,состояние
0,bmw,1400500,2010,x5,142000,бензин,3.0,at,внедорожник,полный,272,не битый
1,subaru,1160000,2015,forester,125000,бензин,2.0,cvt,внедорожник,полный,150,не битый
2,honda,400000,2009,civic,103130,гибрид,1.3,cvt,седан,передний,95,не битый
3,daewoo,85000,2008,matiz,106000,бензин,0.8,mt,хетчбэк,передний,52,не битый
4,lada,150000,2000,4x4 (нива),160000,бензин,1.6,mt,внедорожник,полный,80,не битый
...,...,...,...,...,...,...,...,...,...,...,...,...
4546,subaru,550000,2006,outback,215500,бензин,2.5,mt,универсал,полный,165,не битый
4547,chevrolet,439900,2011,cruze,130000,бензин,1.8,at,седан,передний,141,не битый
4548,vortex,324900,2011,tingo,36000,бензин,1.9,mt,внедорожник,передний,132,не битый
4549,nissan,629000,2008,altima,94510,бензин,2.5,cvt,купе,передний,175,не битый


In [7]:
new_df = make_df(df,df1)

In [8]:
new_df

,название,цена,год,модель,пробег,тип двигателя,объем двигателя,коробка передач,кузов,привод,л.с,состояние
0,porsche,2950000,2013,cayman,46000,бензин,2.7,amt,купе,задний,275,не битый
1,mitsubishi,470000,2000,eclipse,170000,бензин,3.0,at,купе,передний,208,не битый
2,toyota,850000,2001,land cruiser prado,185200,бензин,3.4,mt,внедорожник,полный,185,не битый
3,kia,659000,2017,rio,18300,бензин,1.4,mt,седан,передний,107,не битый
4,renault,580000,2014,duster,200000,бензин,2.0,at,внедорожник,полный,135,не битый
...,...,...,...,...,...,...,...,...,...,...,...,...
6193,subaru,550000,2006,outback,215500,бензин,2.5,mt,универсал,полный,165,не битый
6194,chevrolet,439900,2011,cruze,130000,бензин,1.8,at,седан,передний,141,не битый
6195,vortex,324900,2011,tingo,36000,бензин,1.9,mt,внедорожник,передний,132,не битый
6196,nissan,629000,2008,altima,94510,бензин,2.5,cvt,купе,передний,175,не битый


In [9]:
new_df.to_csv('new_data_frame.csv')